# Simple Token Distribution

A cadCAD sketch exploring an inflationary token model managed and distributed by community contributors.


## Check cadCAD

This cell doesn't do anything, but it does ensure that you know what version of cadCAD you're running. That way if/when you encounter technical difficulties you can tell the community which version of cadCAD you're running. Might save you hours of pain if all you need to do is upgrade to the latest version. 

In [111]:
%pip show cadCAD

Name: cadCAD
Version: 0.4.23
Summary: cadCAD: a differential games based simulation software package for research, validation, and         Computer Aided Design of economic systems
Home-page: https://github.com/cadCAD-org/cadCAD
Author: Joshua E. Jodesty
Author-email: joshua@block.science
License: LICENSE.txt
Location: /usr/local/lib/python3.8/site-packages
Requires: funcy, pathos, fn, pandas
Required-by: cadCAD-diagram
Note: you may need to restart the kernel to use updated packages.


## Import Stuff

These are the libraries you'll need (cadCAD stuff) and that could be useful (python stuff) for your cadCAD model.

In [112]:
# python stuff
import numpy as np
import random as random

# cadCAD stuff
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

## Initial Params

These are the parameters that control the behavior of a system. For example, if you were modeling a Web3 protocol these would be the parameters within a system that token holders could vote to modify. All of the parameters of a protocol that can be modified should be accessible here. That way it's easy to modify and run similations to explore how parameter changes might affect the network.

In [113]:
# the initial state of the cadCAD model
# in a stock and flow model you can think of this as the stocks
genesis_states = {
    # a variable that tracks the total amount of tokens in the system
    'tokens': 1,
    # contributor stats
    # initially this is just token balances
    # in the future it might include skills, effort, time, etc..
    'contributors': {
        i: {'balance': 1} for i in range(1)    
    },
    # the pool of tokens to reward contributors
    'treasury': 0,
    # number of tokens to mint to the treasury 
    'weekly_token_minting': 100,
    'weekly_new_contributors': 2,
    'maximum_effort': 3
}

# The parameters to run the model
# Often these are found towards the end of the file near the cadCAD engine
# but we're putting them here so that you can easily configure and run 
# the notebook without having to scroll back and forth
sim_config_dict = {
    # timesteps: weekly
    'T': range(52), 
    # parallel runs
    'N': 1,
    # more advanced stuff
    #'M': {}
}

## Policy Functions

Policy functions are like flows in stock and flow diagrams. They modify the inputs to state update functions.

We start policy functions with p_ so that they're easier to keep track of when we put them into state update blocks. 

In [114]:
# Mint tokens on a weekly basis (each timestep)
def p_mint_tokens(params, step, sH, s):
    minted_tokens = s['weekly_token_minting']
    return ({'minted_tokens': minted_tokens})

# Do work to earn tokens
def p_do_work(params, step, sH, s):
    # some contributions create more value than others
    impact = random.randint(1, s['maximum_effort'])
    return ({'impact': impact})

# New community contributors
def p_new_community_contributors(params, step, sH, s):
    latest_contributor_key = list(s['contributors'])[-1]
    new_contributors = {i:{'balance': 0} for i in range(latest_contributor_key + 1, latest_contributor_key + s['weekly_new_contributors'])}
    return ({'new_contributors': new_contributors})


## State Update Functions

These functions take in inputs (state variables and policies) and modify the state.

We start policy functions with p_ so that they're easier to keep track of when we put them into state update blocks.

In [115]:
# Update token count based on weekly minting
def s_update_token_count(params, step, sH, s, _input):
    y = 'tokens'
    x = s[y]
    x += _input['minted_tokens']
    return (y, x)

# Put newly minted tokens into the community treasury
def s_mint_to_treasury(params, step, sH, s, _input):
    y = 'treasury'
    x = s[y]
    x += _input['minted_tokens']
    return (y, x)

# Give new contributors accounts
def s_new_contributor_accounts(params, step, sH, s, _input):
    y = 'contributors'
    x = s[y]
    x.update(_input['new_contributors'])
    return (y, x)

# Update community treasury based on contribution rewards
def s_distribute_treasury(params, step, sH, s, _input):
    y = 'treasury'
    x = s[y]
    for i in s['contributors']:
        if x >= _input['impact']:
            x -= _input['impact']
        else:
            break
    return (y, x)

# Distribute rewards to contributors
def s_reward_contributions(params, step, sH, s, _input):
    y = 'contributors'
    x = s[y]
    for i in x:
        if s['treasury'] >= _input['impact']:
            x[i]['balance'] += _input['impact']
        else:
            break
    return (y, x)


## State Update Block

If you're approaching cadCAD from Web3 you can think of the state of the model as something that evolves in blocks. Each block has a set of actions that updates the state. Those transactions then get batched into blocks to be processed together. In cadCAD blocks are called "`partial_state_update_blocks`." As you can see below, this is an array that is very similar to a "block" in a blockchain in that it represents a set of actions to update the state. That state is then updated across many timesteps. This is similar to how the state of a blockchain is updated over many timesteps as new blocks are added.



In [116]:
partial_state_update_blocks = [
    { 
        # mint tokens
        'policies': { 
            'minted_tokens': p_mint_tokens
        },
        # update token count and community treasury
        'variables': {
            'tokens': s_update_token_count,
            'treasury': s_mint_to_treasury
        }
    },
    {
        # add new contributors
        'policies': {
            'new_contributors': p_new_community_contributors
        },
        # give contributors accounts
        'variables': {
            'contributors': s_new_contributor_accounts
        }
    },
    {
        # do work
        'policies': {
            'impact': p_do_work
        },
        # recognize and reward contributors
        'variables': {
            'treasury': s_distribute_treasury,
            'contributors': s_reward_contributions
        }
    }
]

## Running the cadCAD Engine

In [117]:
# imported some addition utilities to help with configuration set-up
exp = Experiment()
c = config_sim(sim_config_dict)
# The configurations above are then packaged into a `Configuration` object
del configs[:]
# dict containing variable names and initial values
exp.append_configs(initial_state=genesis_states, 
                    # dict containing state update functions 
                    partial_state_update_blocks=partial_state_update_blocks, 
                    # preprocessed dictionaries containing simulation parameters
                    sim_configs=c)


In [118]:
%%capture
exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(exec_mode.local_mode)

# pass the configuration object inside an array
simulation = Executor(exec_context=local_mode_ctx, configs=configs) 
# the `execute()` method returns a tuple; its first elements contains the raw results
raw_system_events, tensor_field, sessions = simulation.execute() 

## Data Visualization

This is often half the battle. Not only do need to design and build a cadCAD model, but you need to understand how it's working and be able to effectively communicate that to other people. A picture says a thousand words, thus enter data viz. Getting good at using python data viz libraries is probably the highest leverage thing you can do after you learn the cadCAD basics.

In [119]:
%matplotlib inline
import pandas as pd
simulation_result = pd.DataFrame(raw_system_events)
simulation_result.set_index(['subset', 'run', 'timestep', 'substep'])

tokens  \
subset run timestep substep           
0      1   0        0             1   
           1        1           101   
                    2           101   
                    3           101   
           2        1           201   
...                             ...   
           51       2          5101   
                    3          5101   
           52       1          5201   
                    2          5201   
                    3          5201   

                                                                  contributors  \
subset run timestep substep                                                      
0      1   0        0                                      {0: {'balance': 1}}   
           1        1                                      {0: {'balance': 1}}   
                    2                   {0: {'balance': 1}, 1: {'balance': 0}}   
                    3                   {0: {'balance': 8}, 1: {'balance': 7}}   
           2        1                   {0: {'balance': 8}, 1: {'balance': 7}}   
...                                                                        ...   
           51       2        {0: {'balance': 277}, 1: {'balance': 276}, 2: ...   
                    3        {0: {'balance': 287}, 1: {'balance': 286}, 2: ...   
           52       1        {0: {'balance': 287}, 1: {'balance': 286}, 2: ...   
                    2        {0: {'balance': 287}, 1: {'balance': 286}, 2: ...   
                    3        {0: {'balance': 288}, 1: {'balance': 287}, 2: ...   

                             treasury  weekly_token_minting  \
subset run timestep substep                                   
0      1   0        0               0                   100   
           1        1             100                   100   
                    2             100                   100   
                    3              86                   100   
           2        1             186                   100   
...                               ...                   ...   
           51       2             108                   100   
                    3               8                   100   
           52       1             108                   100   
                    2             108                   100   
                    3              55                   100   

                             weekly_new_contributors  maximum_effort  \
subset run timestep substep                                            
0      1   0        0                              2              10   
           1        1                              2              10   
                    2                              2              10   
                    3                              2              10   
           2        1                              2              10   
...                                              ...             ...   
           51       2                              2              10   
                    3                              2              10   
           52       1                              2              10   
                    2                              2              10   
                    3                              2              10   

                             simulation  
subset run timestep substep              
0      1   0        0                 0  
           1        1                 0  
                    2                 0  
                    3                 0  
           2        1                 0  
...                                 ...  
           51       2                 0  
                    3                 0  
           52       1                 0  
                    2                 0  
                    3                 0  

[157 rows x 7 columns]

simulation_result.plot('timestep', ['tokens', 'treasury'], grid=True, 
        colormap = 'gist_rainbow',
        xticks=list(simulation_result['timestep'].drop_duplicates()), 
        yticks=list(range(1+(simulation_result['tokens']+simulation_result['treasury']).max())))

In [127]:
raw_system_events

balance': 161},
   20: {'balance': 153},
   21: {'balance': 149},
   22: {'balance': 148},
   23: {'balance': 143},
   24: {'balance': 139},
   25: {'balance': 133},
   26: {'balance': 130},
   27: {'balance': 125},
   28: {'balance': 121},
   29: {'balance': 113},
   30: {'balance': 110},
   31: {'balance': 109},
   32: {'balance': 102},
   33: {'balance': 95},
   34: {'balance': 93},
   35: {'balance': 84},
   36: {'balance': 75},
   37: {'balance': 68},
   38: {'balance': 60},
   39: {'balance': 53},
   40: {'balance': 48},
   41: {'balance': 46},
   42: {'balance': 38},
   43: {'balance': 28},
   44: {'balance': 22},
   45: {'balance': 17},
   46: {'balance': 10},
   47: {'balance': 4},
   48: {'balance': 0}},
  'weekly_token_minting': 100,
  'weekly_new_contributors': 2,
  'maximum_effort': 10,
  'simulation': 0,
  'subset': 0,
  'run': 1,
  'substep': 2,
  'timestep': 48},
 {'tokens': 4801,
  'treasury': 2,
  'contributors': {0: {'balance': 267},
   1: {'balance': 266},
   2: {'b